In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import numpy as np

# Flag mode toggle (global variable)
FLAG_MODE = False

class EmojiMinesweeper:
    def __init__(self, width=10, height=10, num_mines=15):
        # Game settings
        self.width = width
        self.height = height
        self.num_mines = min(num_mines, width * height - 1)  # Ensure we don't have too many mines
        
        # Game state
        self.game_over = False
        self.win = False
        self.first_click = True
        
        # Emojis for visualization
        self.emojis = {
            'covered': '🟦',   # Covered cell
            'flag': '🚩',      # Flagged cell
            'mine': '💣',      # Mine
            'explosion': '💥', # Exploded mine
            'empty': '⬜',     # Empty cell
            'numbers': ['⬜', '1️⃣', '2️⃣', '3️⃣', '4️⃣', '5️⃣', '6️⃣', '7️⃣', '8️⃣']  # Number cells
        }
        
        # Initialize the board
        self.initialize_board()
        self.create_widgets()
        
    def initialize_board(self):
        """Initialize the game board."""
        # Create empty board
        self.board = np.zeros((self.height, self.width), dtype=int)  # 0 = no mine, 1 = mine
        self.revealed = np.zeros((self.height, self.width), dtype=bool)  # False = covered, True = revealed
        self.flagged = np.zeros((self.height, self.width), dtype=bool)   # False = not flagged, True = flagged
        
        # Mines will be placed after the first click
    
    def place_mines(self, first_row, first_col):
        """Place mines randomly, avoiding the first clicked cell."""
        # Create a list of all positions
        positions = [(r, c) for r in range(self.height) for c in range(self.width)]
        
        # Remove the first clicked position and its surrounding cells
        safe_positions = []
        for r in range(max(0, first_row-1), min(self.height, first_row+2)):
            for c in range(max(0, first_col-1), min(self.width, first_col+2)):
                safe_positions.append((r, c))
        
        # Remove safe positions from candidate positions
        for pos in safe_positions:
            if pos in positions:
                positions.remove(pos)
        
        # Randomly select positions for mines
        mine_positions = random.sample(positions, self.num_mines)
        
        # Place mines on the board
        for r, c in mine_positions:
            self.board[r, c] = 1
    
    def count_adjacent_mines(self, row, col):
        """Count the number of adjacent mines for a cell."""
        count = 0
        for r in range(max(0, row-1), min(self.height, row+2)):
            for c in range(max(0, col-1), min(self.width, col+2)):
                if (r, c) != (row, col) and self.board[r, c] == 1:
                    count += 1
        return count
    
    def reveal_cell(self, row, col):
        """Reveal a cell and handle game logic."""
        # If game is over or cell is already revealed or flagged, do nothing
        if self.game_over or self.revealed[row, col] or self.flagged[row, col]:
            return
        
        # Handle first click
        if self.first_click:
            self.place_mines(row, col)
            self.first_click = False
        
        # Reveal the cell
        self.revealed[row, col] = True
        
        # Check if mine was hit
        if self.board[row, col] == 1:
            self.game_over = True
            return
        
        # If cell has no adjacent mines, reveal neighboring cells recursively
        if self.count_adjacent_mines(row, col) == 0:
            for r in range(max(0, row-1), min(self.height, row+2)):
                for c in range(max(0, col-1), min(self.width, col+2)):
                    if (r, c) != (row, col) and not self.revealed[r, c]:
                        self.reveal_cell(r, c)
        
        # Check if player has won
        if self.check_win():
            self.game_over = True
            self.win = True
    
    def toggle_flag(self, row, col):
        """Toggle flag on a cell."""
        if not self.game_over and not self.revealed[row, col]:
            self.flagged[row, col] = not self.flagged[row, col]
    
    def check_win(self):
        """Check if player has won."""
        # Player wins if all non-mine cells are revealed
        for r in range(self.height):
            for c in range(self.width):
                if self.board[r, c] == 0 and not self.revealed[r, c]:
                    return False
        return True
    
    def get_cell_display(self, row, col):
        """Get the display emoji for a cell."""
        if not self.revealed[row, col]:
            return self.emojis['flag'] if self.flagged[row, col] else self.emojis['covered']
        elif self.board[row, col] == 1:
            return self.emojis['explosion'] if self.game_over else self.emojis['mine']
        else:
            mines = self.count_adjacent_mines(row, col)
            return self.emojis['numbers'][mines]
    
    def create_widgets(self):
        """Create the game widgets."""
        # Game status label
        self.status_label = widgets.Label(value="Game in progress. Use the mode toggle to switch between reveal and flag.")
        
        # Create grid of button widgets
        self.buttons = []
        for r in range(self.height):
            row_buttons = []
            for c in range(self.width):
                button = widgets.Button(
                    description=self.emojis['covered'],
                    layout=widgets.Layout(width='40px', height='40px'),
                    disabled=False
                )
                # Capture current position for the click handlers
                pos = (r, c)
                button.on_click(lambda b, pos=pos: self.handle_click(pos))
                row_buttons.append(button)
            self.buttons.append(row_buttons)
        
        # Arrange buttons in a grid
        self.grid = widgets.GridBox(
            children=[button for row in self.buttons for button in row],
            layout=widgets.Layout(
                grid_template_columns=f"repeat({self.width}, 40px)",
                grid_template_rows=f"repeat({self.height}, 40px)",
                grid_gap="0px"
            )
        )
        
        # Flag mode toggle button
        self.flag_button = widgets.Button(
            description="Mode: 🔍 REVEAL",
            layout=widgets.Layout(width='150px', height='30px')
        )
        self.flag_button.on_click(self.toggle_flag_mode)
        
        # Reset button
        self.reset_button = widgets.Button(
            description="New Game",
            layout=widgets.Layout(width='150px', height='30px')
        )
        self.reset_button.on_click(lambda b: self.reset_game())
        
        # Button container
        button_container = widgets.HBox([self.flag_button, self.reset_button])
        
        # Game info and flags count
        self.info_label = widgets.Label(value=f"Mines: {self.num_mines} | Flags: 0")
        
        # Container for the game
        self.game_container = widgets.VBox([
            self.status_label,
            self.info_label,
            self.grid,
            button_container
        ])
    
    def handle_click(self, pos):
        """Handle cell click based on current mode."""
        global FLAG_MODE
        r, c = pos
        
        if FLAG_MODE:
            self.toggle_flag(r, c)
        else:
            self.reveal_cell(r, c)
            
        self.update_display()
    
    def toggle_flag_mode(self, b):
        """Toggle between flag and reveal modes."""
        global FLAG_MODE
        FLAG_MODE = not FLAG_MODE
        
        if FLAG_MODE:
            self.flag_button.description = "Mode: 🚩 FLAG"
            self.status_label.value = "Flag mode: Click on cells to place or remove flags."
        else:
            self.flag_button.description = "Mode: 🔍 REVEAL"
            self.status_label.value = "Reveal mode: Click on cells to reveal them."
    
    def update_display(self):
        """Update the display of all cells."""
        for r in range(self.height):
            for c in range(self.width):
                self.buttons[r][c].description = self.get_cell_display(r, c)
        
        # Update game status
        flag_count = np.sum(self.flagged)
        self.info_label.value = f"Mines: {self.num_mines} | Flags: {flag_count}"
        
        if self.game_over:
            if self.win:
                self.status_label.value = "🎉 You won! 🎉"
            else:
                self.status_label.value = "💥 Game over! You hit a mine. 💥"
    
    def reset_game(self):
        """Reset the game."""
        global FLAG_MODE
        self.game_over = False
        self.win = False
        self.first_click = True
        
        # Reset flag mode to reveal
        FLAG_MODE = False
        self.flag_button.description = "Mode: 🔍 REVEAL"
        
        self.initialize_board()
        self.status_label.value = "Reveal mode: Click on cells to reveal them."
        self.update_display()
    
    def display(self):
        """Display the game."""
        display(self.game_container)

In [ ]:
# In a new cell
# Create and display the game
game = EmojiMinesweeper(width=10, height=8, num_mines=12)
game.display()

# Instructions
print("How to play:")
print("• Use the mode toggle button to switch between REVEAL and FLAG modes")
print("• In REVEAL mode, click cells to reveal them")
print("• In FLAG mode, click cells to place or remove flags")
print("• Clear all non-mine cells to win!")
print("• The numbers show how many mines are adjacent to the cell")